In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/to_predict.csv
/kaggle/input/cologne_combined_data_left.csv
/kaggle/input/cologne_combined_data.csv
/kaggle/input/RhineWaterLevelData/team9submission.csv
/kaggle/input/RhineWaterLevelData/weather_stations.csv
/kaggle/input/RhineWaterLevelData/station6335050_water_level.csv
/kaggle/input/RhineWaterLevelData/weather_data_cleaned.csv
/kaggle/input/RhineWaterLevelData/river_data.csv
/kaggle/input/RhineWaterLevelData/reference.csv
/kaggle/input/RhineWaterLevelData/river_stations.csv
/kaggle/input/RhineWaterLevelData/weather_data.csv


In [2]:
df = pd.read_csv('../input/cologne_combined_data_left.csv')
df.date = pd.to_datetime(df.date, format='%Y-%m-%d')
df['day_x'] = df.date.dt.day
df = df.set_index('date')
df.head()

,station_no,discharge,water_level,year_x,month_x,delta1,delta2,delta3,STATIONS_ID,RS,RSF,SH_TAG,NSH_TAG,year_y,month_y,day_x
date,,,,,,,,,,,,,,,,
1965-01-01,6335050,1320.0,219.0,1965,1,NaN,NaN,NaN,2968.0,1.2,1.0,0.0,0.0,1965.0,1.0,1
1965-01-02,6335050,1650.0,272.0,1965,1,-53.0,NaN,NaN,2968.0,0.3,1.0,0.0,0.0,1965.0,1.0,2
1965-01-03,6335050,1950.0,315.0,1965,1,-43.0,-96.0,NaN,2968.0,0.1,1.0,0.0,0.0,1965.0,1.0,3
1965-01-04,6335050,1870.0,304.0,1965,1,11.0,-32.0,-85.0,2968.0,0.0,7.0,0.0,0.0,1965.0,1.0,4
1965-01-05,6335050,1870.0,304.0,1965,1,0.0,11.0,-32.0,2968.0,1.2,1.0,0.0,0.0,1965.0,1.0,5


In [3]:
df_encoded = pd.get_dummies(df, columns=['year_x', 'month_x', 'day_x'])
df_encoded = df_encoded.fillna(0)
df_encoded.head()

,station_no,discharge,water_level,delta1,delta2,delta3,STATIONS_ID,RS,RSF,SH_TAG,...,day_x_22,day_x_23,day_x_24,day_x_25,day_x_26,day_x_27,day_x_28,day_x_29,day_x_30,day_x_31
date,,,,,,,,,,,,,,,,,,,,,
1965-01-01,6335050,1320.0,219.0,0.0,0.0,0.0,2968.0,1.2,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1965-01-02,6335050,1650.0,272.0,-53.0,0.0,0.0,2968.0,0.3,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1965-01-03,6335050,1950.0,315.0,-43.0,-96.0,0.0,2968.0,0.1,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1965-01-04,6335050,1870.0,304.0,11.0,-32.0,-85.0,2968.0,0.0,7.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1965-01-05,6335050,1870.0,304.0,0.0,11.0,-32.0,2968.0,1.2,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train, test = train_test_split(df_encoded, test_size=0.2, shuffle=False)

train_X = train.values[:-1, 2:]
train_Y = train.delta1.values[1:]
train_Y2 = train.delta2.values[1:]
train_Y3 = train.delta3.values[1:]

print(train_X.shape, train_Y.shape)
print(train_X[:10, :])
print(train_Y[:10])

(11912, 102) (11912,)
[[219.   0.   0. ...   0.   0.   0.]
 [272. -53.   0. ...   0.   0.   0.]
 [315. -43. -96. ...   0.   0.   0.]
 ...
 [272.   0.  18. ...   0.   0.   0.]
 [299. -27. -27. ...   0.   0.   0.]
 [370. -71. -98. ...   0.   0.   0.]]
[ -53.  -43.   11.    0.   14.   18.    0.  -27.  -71. -114.]


In [5]:
test_X = test.values[:-1, 2:]
test_Y = test.delta1.values[1:]
test_Y2 = test.delta2.values[1:]
test_Y3 = test.delta3.values[1:]

print(test_X.shape, test_Y.shape)


(2978, 102) (2978,)


In [6]:
def get_previous_days_data(station_no, date):
    date_prev = date - pd.Timedelta(days=1)
    res = df_encoded[(df_encoded.station_no == station_no) & (df_encoded.index == date_prev)]
    return res

In [7]:
regressors = []
for (trainY, testY) in zip([train_Y, train_Y2, train_Y3], [test_Y, test_Y2, test_Y3]):
    regr = RandomForestRegressor(n_estimators=100)
    regr.fit(train_X, trainY)
    predictions = regr.predict(test_X)
    print(r2_score(testY, predictions))
    regressors.append(regr)

0.3549192885465805
0.7970491906924719
0.8987159854401178


In [8]:
targets = pd.read_csv('../input/to_predict.csv')
targets.date = pd.to_datetime(targets.date, format='%Y-%m-%d')

In [9]:
delta1 = []
delta2 = []
delta3 = []
for (i, row) in targets.iterrows():
    if i%3 != 0:
        continue
    print(row)
    if row.station_no == '6335060':
        #TODO: We do not yet have the data for this station
        continue
    input_X = get_previous_days_data(row.station_no, row.date)
    input_X = input_X.values[:, 2:]
    delta1.append(regressors[0].predict(input_X))    
    delta2.append(regressors[1].predict(input_X))
    delta3.append(regressors[2].predict(input_X))

station_no                6335050
date          1965-04-17 00:00:00
Name: 0, dtype: object
station_no                6335050
date          1965-08-01 00:00:00
Name: 3, dtype: object
station_no                6335050
date          1965-11-15 00:00:00
Name: 6, dtype: object
station_no                6335050
date          1966-03-01 00:00:00
Name: 9, dtype: object
station_no                6335050
date          1966-06-15 00:00:00
Name: 12, dtype: object
station_no                6335050
date          1966-09-29 00:00:00
Name: 15, dtype: object
station_no                6335050
date          1967-01-13 00:00:00
Name: 18, dtype: object
station_no                6335050
date          1967-04-29 00:00:00
Name: 21, dtype: object
station_no                6335050
date          1967-08-13 00:00:00
Name: 24, dtype: object
station_no                6335050
date          1967-11-27 00:00:00
Name: 27, dtype: object
station_no                6335050
date          1968-03-12 00:00:00
Name: 30, dtype:

station_no                6335050
date          1991-12-29 00:00:00
Name: 276, dtype: object
station_no                6335050
date          1992-04-13 00:00:00
Name: 279, dtype: object
station_no                6335050
date          1992-07-28 00:00:00
Name: 282, dtype: object
station_no                6335050
date          1992-11-11 00:00:00
Name: 285, dtype: object
station_no                6335050
date          1993-02-25 00:00:00
Name: 288, dtype: object
station_no                6335050
date          1993-06-11 00:00:00
Name: 291, dtype: object
station_no                6335050
date          1993-09-25 00:00:00
Name: 294, dtype: object
station_no                6335050
date          1994-01-09 00:00:00
Name: 297, dtype: object
station_no                6335050
date          1994-04-25 00:00:00
Name: 300, dtype: object
station_no                6335050
date          1994-08-09 00:00:00
Name: 303, dtype: object
station_no                6335050
date          1994-11-23 00:00:00
Na

ValueError: Found array with 0 sample(s) (shape=(0, 102)) while a minimum of 1 is required.